In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, to_date, date_format

# Initialize Spark session
spark = SparkSession.builder.appName("Retail").getOrCreate()

# Load datasets with header and infer schema
sales_df = spark.read.csv("sales.csv", header=True, inferSchema=True)
inventory_df = spark.read.csv("inventory.csv", header=True, inferSchema=True)
products_df = spark.read.csv("products.csv", header=True, inferSchema=True)

# Convert sale_date in sales to date type
sales_df = sales_df.withColumn("sale_date", to_date("sale_date"))

# Join sales with inventory on product_id
sales_inventory_df = sales_df.join(inventory_df, on="product_id", how="inner")

# Join the above result with products to add product category
full_df = sales_inventory_df.join(products_df, on="product_id", how="inner")

# Extract year-month from sale_date for monthly trend analysis
full_df = full_df.withColumn("year_month", date_format("sale_date", "yyyy-MM"))

# Aggregate total sales by region, category, and year_month
monthly_sales_trends = full_df.groupBy("region", "category", "year_month") \
                             .agg(sum("quantity_sold").alias("total_sales")) \
                             .orderBy("region", "category", "year_month")

# Show sample results
monthly_sales_trends.show(30, truncate=False)

# Export aggregated results to CSV and Parquet
monthly_sales_trends.write.csv("output/monthly_sales_trends.csv", header=True, mode="overwrite")
monthly_sales_trends.write.parquet("output/monthly_sales_trends.parquet", mode="overwrite")

# Stop Spark session
spark.stop()


+------+-----------+----------+-----------+
|region|category   |year_month|total_sales|
+------+-----------+----------+-----------+
|East  |Accessories|2025-10   |15         |
|East  |Electronics|2025-10   |7          |
|North |Electronics|2025-10   |8          |
|North |Furniture  |2025-10   |2          |
|South |Accessories|2025-10   |10         |
|South |Electronics|2025-10   |4          |
|South |Furniture  |2025-10   |8          |
|West  |Accessories|2025-10   |10         |
|West  |Electronics|2025-10   |7          |
+------+-----------+----------+-----------+

